In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2025 United Kingdom Research and Innovation
#  Copyright 2025 The University of Manchester
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# Authors:
# Evan Kiely (Warwick Manufacturing Group, University of Warwick)
# Edited by Hannah Robarts (STFC - UKRI)

# TescanDataReader Demo

Note: this demo is a work in progress and was created for the purpose of demonstrating a contibution for the reader hackathon!

The TescanDataReader in its current state allows:

- Reading of geometry
- Reading of dark fields
- Reading of flat fields
- Reading of projections 
    - Reading of normalised and unnormalised data
    - Reading Flipped Data
    - Cropping Data




## CIL Version

This notebook was developed using CIL v25.0.0

Data Comes as Tiff files. Data acquisition settings is stored in the Acquisiton Settings XRE.txt file

## Dataset
The dataset is a CT scan of a single material object with closed pores collected by Evelien Zwanenburg and Jay Warnett at WMG on a Tescan Unitom XL. The dataset can be downloaded from https://zenodo.org/records/14993616

Update this filepath to where you have saved the dataset:

In [ ]:
file_path = r'/mnt/share/stfc.ac.uk/tomography/hackathon-11-25/tescan_close_pore/acq_data/Acquisition settings XRE.txt'

## Loading Geometry

In [ ]:
from readers.TescanDataReader import TescanDataReader
from cil.utilities.display import show2D, show_geometry

Create a `TescanDataReader` object and give it the file_path

In [ ]:
reader = TescanDataReader(file_path)

First we just get the geometry and visualise and print it to check it looks right

In [ ]:
acq_geom = reader.get_geometry()
show_geometry(acq_geom)
print(acq_geom)

## Loading Projections, Flat and Dark Fields

Next we load the dataset and use `show2D` to look at a single projection

In [ ]:
data = reader.read()

show2D(data, origin='upper-left')

The loaded data is normalised by default (we can turn this off by specifing `TescanDataReader(filename,normalise=False)`). If we want to check the dark and flat files we use the `load_darkfield()` and `load_flatfield()` methods.

In [ ]:
dark_field = reader.load_darkfield()
flat_field = reader.load_flatfield()

show2D([dark_field, flat_field], ['Dark field', 'Flat field'], origin='upper-left')

# Pre-processing and Reconstruction

In [ ]:
from cil.processors import TransmissionAbsorptionConverter, Slicer, CentreOfRotationCorrector
from cil.recon import FDK
import numpy as np

Now we run some pre-processing steps on the data.

First we use the Beer-Lambert law to convert from transmission to absorption. 

In [ ]:
data_abs = TransmissionAbsorptionConverter()(data)
show2D([data, data_abs], ['Transmission data', 'Absorption data'])

Run a reconstruction using CIL FDK

In [ ]:
recon = FDK(data_abs).run()
show2D(recon)

Next we try to optimise the centre of rotation position

In [ ]:
print(f"Centre of rotation before = {data_abs.geometry.get_centre_of_rotation(distance_units='pixels')}")
data_abs = CentreOfRotationCorrector.image_sharpness(backend='tigre', search_range=100, tolerance=0.1)(data_abs)
print(f"Centre of rotation after = {data_abs.geometry.get_centre_of_rotation(distance_units='pixels')}")

Check if this has an impact on the reconstruction

In [ ]:
recon_abs = FDK(data_abs).run(verbose=False)
show2D([recon.array[375,180:420,300:600], recon_abs.array[375,180:420,300:600]],
       ['Before centre of rotation correction', 'After centre of rotation correction'])

# Reader arguments

Try reading a subset of the data using the `roi` argument.

First we reduce the horizontal and vertical range

In [ ]:
data_binned = TescanDataReader(file_path, roi={'horizontal':(30,710,1), 'vertical':(30,700,1)}).read()
data_binned = TransmissionAbsorptionConverter()(data_binned)
show2D([data_abs, data_binned])

Next we try binning the data using `roi={'horizontal':(0,-1,10), 'vertical':(0,-1,10)}` to specify the full range in vertical and horizontal but binned 10x.

In [ ]:
file_path = r'/mnt/share/stfc.ac.uk/tomography/hackathon-11-25/tescan_close_pore/acq_data/Acquisition settings XRE.txt'
data_binned = TescanDataReader(file_path, roi={'horizontal':(0,-1,10), 'vertical':(0,-1,3)},
                               normalise=True).read()
data_binned = TransmissionAbsorptionConverter()(data_binned)
show2D([data_abs, data_binned])